# FOP Demonstator Sheet

## Converting Excel to CSV

For this entire notebook, you need to manually paste the excel locations file by file into here. If you have multiple files you just paste it one by one.
If you know how to put all files into a directory and use just one loop no matter the number of files, do send in a PR.

In [47]:
import pandas as pd

file1 = pd.read_excel (r'WIX1002_AttendanceSession_435577_2023112-1646.xlsx')
file1.to_csv (r'attn1.csv', index = None, header=True)

file2 = pd.read_excel (r'WIX1002_AttendanceSession_435578_2023112-1646.xlsx')
file2.to_csv (r'attn2.csv', index = None, header=True)

file3 = pd.read_excel (r'WIX1002_AttendanceSession_435579_2023112-1646.xlsx')
file3.to_csv (r'attn3.csv', index = None, header=True)

file4 = pd.read_excel (r'courseid_151_participants (1).xlsx')
file4.to_csv (r'students.csv', index = None, header=True)

## CSV overview

In [48]:
oct12 = pd.read_csv('attn1.csv')
oct19 = pd.read_csv('attn2.csv')
oct26 = pd.read_csv('attn3.csv')
students = pd.read_csv('students.csv')

oct12.rename(columns={'Course': 'ID number'}, inplace=True)
oct12.rename(columns={'WIX1002 FUNDAMENTALS OF PROGRAMMING' : 'Name'}, inplace=True)
oct12.rename(columns={'Unnamed: 2': 'Email address'}, inplace=True)
oct12.rename(columns={'Unnamed: 3': 'Status'}, inplace=True)

oct19.rename(columns={'Course': 'ID number'}, inplace=True)
oct19.rename(columns={'WIX1002 FUNDAMENTALS OF PROGRAMMING' : 'Name'}, inplace=True)
oct19.rename(columns={'Unnamed: 2': 'Email address'}, inplace=True)
oct19.rename(columns={'Unnamed: 3': 'Status'}, inplace=True)

oct26.rename(columns={'Course': 'ID number'}, inplace=True)
oct26.rename(columns={'WIX1002 FUNDAMENTALS OF PROGRAMMING' : 'Name'}, inplace=True)
oct26.rename(columns={'Unnamed: 2': 'Email address'}, inplace=True)
oct26.rename(columns={'Unnamed: 3': 'Status'}, inplace=True)

oct12.head()

,ID number,Name,Email address,Status,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Group,All participants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Session,12-10-2023 10:00AM - 01:00PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ID number,Name,Email address,Status,Time Taken,Latitude,Longitude,Latitude Difference,Longitude Difference,Device,Remark
4,22002566,ABDUL ALIFF AMMAR BIN ABDUL KHALID,22002566@siswa.um.edu.my,Absent,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# ONLY RUN THIS BLOCK ONCE.
# If accidentally ran more than once, run from the top again.

# drop first 4 rows
oct12 = oct12.iloc[4:]
oct19 = oct19.iloc[4:]
oct26 = oct26.iloc[4:]

# keep the first 4 columns only
oct12 = oct12.iloc[:, :4]
oct19 = oct19.iloc[:, :4]
oct26 = oct26.iloc[:, :4]

# drop first row of student (Dr Unaizah)
students = students.iloc[1:]

# sort all dataframes by ID number
oct12 = oct12.sort_values(by=['ID number'])
oct19 = oct19.sort_values(by=['ID number'])
oct26 = oct26.sort_values(by=['ID number'])
students = students.sort_values(by=['ID number'])

oct12.head()

,ID number,Name,Email address,Status
227,17203558,MUHAMMAD AMEER HAKIM BIN HARMI SAZLAN,17203558@siswa.um.edu.my,Absent
230,22000533,MUHAMMAD AMIR BIN MAT RAZI,22000533@siswa.um.edu.my,Absent
15,22000628,AHMAD DANIAL ADLI BIN MOHD ZAKI,22000628@siswa.um.edu.my,Absent
431,22000741,WAN NUR SYAHIRAH BINTI WAN AZIZ,22000741@siswa.um.edu.my,Absent
43,22000822,ARIFF IRFAN BIN CHE HAMZAH,22000822@siswa.um.edu.my,Absent


In [50]:
students.head()

,First name,Surname,ID number,Email address
4,MUHAMMAD SYAHMIE BIN MOHD.AFFANDI,.,22001043,22001043@siswa.um.edu.my
21,NUR SANJIDA BINTI MD TITU,.,22001044,22001044@siswa.um.edu.my
6,AKMAL HARIZ BIN AZIDI,.,22001279,22001279@siswa.um.edu.my
19,TENGKU MOHAMAD HAIKAL BIN TENGKU ABDULLAH,.,22001741,22001741@siswa.um.edu.my
1,ADIB RUSYAIDI BIN MOHD ZAKI,.,22001821,22001821@siswa.um.edu.my


In [51]:
# Keep only students present in students dataframe from oct12, oct19, oct26

oct12 = oct12[oct12['ID number'].isin(students['ID number'])]
oct19 = oct19[oct19['ID number'].isin(students['ID number'])]
oct26 = oct26[oct26['ID number'].isin(students['ID number'])]

oct12.head(50)

,ID number,Name,Email address,Status
266,22001043,MUHAMMAD SYAHMIE BIN MOHD.AFFANDI,22001043@siswa.um.edu.my,Absent
313,22001044,NUR SANJIDA BINTI MD TITU,22001044@siswa.um.edu.my,Absent
25,22001279,AKMAL HARIZ BIN AZIDI,22001279@siswa.um.edu.my,Absent
404,22001741,TENGKU MOHAMAD HAIKAL BIN TENGKU ABDULLAH,22001741@siswa.um.edu.my,Absent
9,22001821,ADIB RUSYAIDI BIN MOHD ZAKI,22001821@siswa.um.edu.my,Absent
264,22001866,MUHAMMAD NUR FITRI BIN FAIZAN,22001866@siswa.um.edu.my,Absent
16,22002089,AHMAD FAHIIM BIN MOHAMAD KAMAL,22002089@siswa.um.edu.my,Absent
241,22002100,MUHAMMAD DANIEL BIN MOHD NAZRI,22002100@siswa.um.edu.my,Absent
8,22002146,ADAM BIN RAZALI,22002146@siswa.um.edu.my,Absent
125,22002203,IMAN SOFFEA BINTI JAMAL,22002203@siswa.um.edu.my,Absent


In [52]:
# Create a new dataframe with only the ID number and Name columns

students = students[['ID number', 'First name']]
students.head()

,ID number,First name
4,22001043,MUHAMMAD SYAHMIE BIN MOHD.AFFANDI
21,22001044,NUR SANJIDA BINTI MD TITU
6,22001279,AKMAL HARIZ BIN AZIDI
19,22001741,TENGKU MOHAMAD HAIKAL BIN TENGKU ABDULLAH
1,22001821,ADIB RUSYAIDI BIN MOHD ZAKI


In [53]:
# append attendance of oct12 to the end of students, with column title 'oct12'

students['oct12'] = oct12['Status'].tolist()
students['oct19'] = oct19['Status'].tolist()
students['oct26'] = oct26['Status'].tolist()
students.head(50)

,ID number,First name,oct12,oct19,oct26
4,22001043,MUHAMMAD SYAHMIE BIN MOHD.AFFANDI,Absent,Present,Present
21,22001044,NUR SANJIDA BINTI MD TITU,Absent,Present,Present
6,22001279,AKMAL HARIZ BIN AZIDI,Absent,Present,Present
19,22001741,TENGKU MOHAMAD HAIKAL BIN TENGKU ABDULLAH,Absent,Present,Present
1,22001821,ADIB RUSYAIDI BIN MOHD ZAKI,Absent,Present,Present
20,22001866,MUHAMMAD NUR FITRI BIN FAIZAN,Absent,Present,Present
2,22002089,AHMAD FAHIIM BIN MOHAMAD KAMAL,Absent,Present,Present
23,22002100,MUHAMMAD DANIEL BIN MOHD NAZRI,Absent,Present,Present
18,22002146,ADAM BIN RAZALI,Absent,Present,Present
3,22002203,IMAN SOFFEA BINTI JAMAL,Absent,Present,Present


In [54]:
# output to csv

students.to_csv('final.csv', index=False)